In [20]:
import pandas as pd
import numpy as np 
import random
from sklearn.decomposition import PCA
from tqdm import tqdm
import plotly.express as px 
import plotly.io as pio 
pio.renderers.default = "browser"
import ast

In [40]:
    n = 6
    w = [[(100/n)/100]*n]
    for i in range(1,20000):
        weights = [random.random() for _ in range(n)]
        sum_weights = sum(weights)
        weights = [1*w/sum_weights for w in weights]
        w.append(list(np.round(weights,2)))
    weights_new = []
    for i in w:
        if i not in weights_new:
            weights_new.append(i)

# Rolling Periods

In [43]:
#Finding the initial portoflio from desired investemnt universe 

def calculate_pdi(num_assets, tickers, weekly_returns): 
        
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/52))-1
            
            return(Result)

        pca = PCA()
        PDI_dict = {}
        samples = []
        for number in [num_assets]:
            for i in range(1,20000):
                #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
                samples.append(random.sample(list(tickers),number))
        seen = set()
        samples_mini = [x for x in samples if frozenset(x) not in seen and not seen.add(frozenset(x))]


        
        for i,y in tqdm(zip(samples_mini,range(1,len(samples_mini)+1))):
            #prog = int(y/len(samples_mini)*100)
            #progress_bar.progress(prog)
            #status_text.text("{}% Complete".format(prog))
            n_assets = len(i)
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            ann_ret = meanRetAn(list(port_weekly_return))
            an_cov = weekly_returns[i].cov()
            port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
            corr_matrix = np.array(weekly_returns[i].corr())
            principalComponents = pca.fit(corr_matrix)
            PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
            
            PDI_dict[y] = {}
            PDI_dict[y]["PDI_INDEX"] = PDI
            PDI_dict[y]["# of Assets"] = len(i)
            PDI_dict[y]["Assets"] = i
            PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
            PDI_dict[y]["Annual Return"] = ann_ret
            PDI_dict[y]["Annual STD"] = port_std
        

            


        PDI_DF = pd.DataFrame(PDI_dict).T
        PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
        PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
        PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
        PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
        PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
        PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)

        return PDI_DF



############################################################## Trading Strategy #################################################################################
# Trading algorithm that uses the portfolio chosen, and allocated weights accordingly
def calculate_pdi_weights( returns,return_mean_range): 

    n = len(returns.columns)
    eq = [1/n]*n
    w = []
    w.append(eq)
    for i in range(1,20000):
        weights = [random.random() for _ in range(n)]
        sum_weights = sum(weights)
        weights = [1*w/sum_weights for w in weights]
        w.append(list(np.round(weights,2)))
    weights_new = []
    for i in w:
        if i not in weights_new:
            weights_new.append(i)


    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/return_mean_range))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for y,num in tqdm(zip(weights_new, range(0,len(weights_new),1))):
        
        port_ret  = returns.mul(y,axis=1).sum(axis=1)

        ann_ret = meanRetAn(list(port_ret))
        an_cov = returns.cov()
        port_std = np.sqrt(np.dot(np.array(y).T, np.dot(an_cov, y)))*np.sqrt(return_mean_range)
        corr_matrix = np.array(returns.mul(y).cov())
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1

        PDI_dict[num ] = {}
        PDI_dict[num ]["PDI_INDEX"] = PDI
        PDI_dict[num ]["# of Assets"] = len(y)
        PDI_dict[num ]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[num ]["Annual Return"] = ann_ret
        PDI_dict[num ]["weights"] = y
        PDI_dict[num ]["Annual STD"] = port_std

    df = pd.DataFrame(PDI_dict).T
    df["PDI_INDEX"] = df["PDI_INDEX"].astype(float)
    df["Sharpe Ratio"] = df["Sharpe Ratio"].astype(float)
    df["Annual Return"] = df["Annual Return"].astype(float)
    df["Annual STD"] = df["Annual STD"].astype(float)

    return df



############################################################## Trading Strategy #################################################################################
# Trading algorithm that finds new portfolios each quarter
def pca_per_weights_rolling(return_data, portfolio, interval, ret_range_mean,pdi_max_train):
        data = return_data.copy() # data containing weekly returns
        tickers = list(data.columns)
        data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
        weeks_list = data[data.index.year > 2015].index # grabbing all index dates
        data.index = data.index.to_period(interval) # converting the index to quarterly sets
        periods = data.index.unique() # taking the unique quarters to loop

        
        list_range = [] # saving rolling periods
        list_period = periods[4:] # periods of return
        for i in range(1,21): 
            list_range.append(periods[i:4+i])

        #print(periods)
        first_period = list_period[0] # the first period of the time frame
        remaining_periods = list_period[1:] # the remianing periods for returns calculations
        pdi_rolling_periods = list_range[:-1] # all periods minus the last

        ########################################  Function for pdi ########## ########## ########## ########## ########### #########  
        def pdi_period(returns, period, weights):
            pca = PCA()
            corr_matrix = np.array(returns.loc[period].mul(weights).cov())
            principalComponents = pca.fit(corr_matrix)
            return 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
        ########## ########## ########## ##########  Mean Annual Return Function ########## ########## ########## ########## ########## 
        def meanRetAn(data):             
            Result = 1
            
            for i in data:
                Result *= (1+i)
                
            Result = Result**(1/float(len(data)/ret_range_mean))-1
            
            return(Result)

        ########## ########## ########## ##########  Portfolio Return ########## ########## ########## ########## ########## ########## 
        def port_ret(returns, period, weights): # function for calculating returns
            portfolio_weights_ew = weights
            port_return = returns.loc[period].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            return  port_return

        pdi_performance_w = [] #saving performance of max pdi
        sharpe_performance_w= [] #saving performance of optimal portfolio of max portfolio
        equal_performance = [] #saving equal weight portflio performance 
        sharpe_2_performance_w =[]


        weights_pdi_performance_w = [] #weights for max pdi allocation over time
        weights_sharpe_performance_w= [] #weights for max sharpe ratio allocation over time
        weights_equal_performance = [] # weights for equal portfolio over time "same all periods"
        weights_sharpe_2_performance_w= []
        periods_weights = [] # saving periods for the weights allocation

        pdi_weights_pdi_performance_w = [] #pdi for max pdi allocation over time
        pdi_weights_sharpe_performance_w= [] #pdi for max sharpe ratio allocation over time
        pdi_weights_equal_performance = [] #pdi for equal portfolio over time "same all periods"
        pdi_weights_sharpe_2_performance_w= []



        
        assets = [] # store asstes for all periods
        assets.append(portfolio) # appending portfolio
        ############################################################ Calculate first period ################################################################################
        print(first_period)
        periods_weights.append(first_period)
        first_period_df = calculate_pdi_weights( returns = data[data.index.year == 2015],return_mean_range = 52)

        ################################################## Taking the higest PDI ###########################################################################################

        id = first_period_df["PDI_INDEX"].idxmax()
        port_max_pdi_weights = first_period_df["weights"][id] # getting weights for period
        port_max_pdi_weights_pdi = first_period_df["PDI_INDEX"][id] # getting weights for period
        
        weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
        port_max_ret_period = port_ret(data, first_period,port_max_pdi_weights)  # calculating return for periods 
        pdi_performance_w.extend(port_max_ret_period) # saving return 
        pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        id_sharpe = first_period_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_weights_1 = first_period_df["weights"][id_sharpe] # getting weights for period
        port_max_sharpe_weights_pdi = first_period_df["PDI_INDEX"][id_sharpe] # getting weights for period

        weights_sharpe_performance_w.append(port_max_sharpe_weights_1)   # saving weights for periods
        port_max_ret_period_sharpe = port_ret(data, first_period,port_max_sharpe_weights_1)  # calculating return for periods 
        sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
        pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period


        ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
        if len(first_period_df[first_period_df["PDI_INDEX"] > 2]) == 0:
            mini_df = first_period_df.copy()
        else:
            mini_df = first_period_df[first_period_df["PDI_INDEX"] > 2].copy()

        id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
        port_max_sharpe_2_weights = mini_df["weights"][id_sharpe_2] # getting weights for period
        port_max_sharpe_2_weights_pdi = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

        weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights)   # saving weights for periods
        port_max_ret_period_sharpe_2 = port_ret(data, first_period,port_max_sharpe_2_weights)  # calculating return for periods 
        sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
        pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_2_weights_pdi) # calculating pdi for first period

        ################################################################## Equal Weigths Portoflio ##########################################################################

        equal_weights = first_period_df.iloc[0]["weights"] # getting weights for period
        

        weights_equal_performance.append(equal_weights)  # saving weights for periods
        port_max_ret_period_equal = port_ret(data, first_period,equal_weights) # calculating return for periods 
        equal_performance.extend(port_max_ret_period_equal) # saving return 
        pdi_weights_equal_performance.append(pdi_max_train) # calculating pdi for first period


        ######################################################## Calculation of portfolio perfomnce #############################################################################

        for init_time, next_time in zip(pdi_rolling_periods, remaining_periods):
            ############ Portfolio Creation ##############################
            print("Rolling range for calculatio: {} - period of return: {}".format(init_time, next_time))
            PDI_DF = calculate_pdi_weights(returns = data.loc[init_time].dropna(axis=1), return_mean_range = ret_range_mean)
            periods_weights.append(next_time) # saving first period
            assets.append(portfolio) # appending portfolio

            ################################################## Taking the higest PDI ##########################################################################

            id = PDI_DF["PDI_INDEX"].idxmax()
            port_max_pdi_weights = PDI_DF["weights"][id] # getting weights for period
            port_max_pdi_weights_pdi = PDI_DF["PDI_INDEX"][id]
            
            weights_pdi_performance_w.append(port_max_pdi_weights) # saving weights for period
            port_max_ret_period = port_ret(data, next_time,port_max_pdi_weights)  # calculating return for periods 
            pdi_performance_w.extend(port_max_ret_period) # saving return 
            pdi_weights_pdi_performance_w.append(port_max_pdi_weights_pdi) # calculating pdi for first period

            ################################################## Taking the higest Sharpe Ration - PDI ##########################################################################
            id_sharpe = PDI_DF["Sharpe Ratio"].idxmax()
            port_max_sharpe_weights = PDI_DF["weights"][id_sharpe] # getting weights for period
            port_max_sharpe_weights_pdi = PDI_DF["PDI_INDEX"][id_sharpe] # getting weights for period

            weights_sharpe_performance_w.append(port_max_sharpe_weights)   # saving weights for periods
            port_max_ret_period_sharpe = port_ret(data, next_time,port_max_sharpe_weights)  # calculating return for periods 
            sharpe_performance_w.extend(port_max_ret_period_sharpe) # saving return 
            pdi_weights_sharpe_performance_w.append(port_max_sharpe_weights_pdi) # calculating pdi for first period

            ################################################## Taking the higest Sharpe Ration - PDI above 2 ##########################################################################
            if len(PDI_DF[PDI_DF["PDI_INDEX"] > 2]) == 0:
                mini_df = PDI_DF.copy()
            else:
                mini_df = PDI_DF[PDI_DF["PDI_INDEX"] > 2].copy()

            id_sharpe_2 = mini_df["Sharpe Ratio"].idxmax()
            port_max_sharpe_weights_2 = mini_df["weights"][id_sharpe_2] # getting weights for period
            port_max_sharpe_weights_pdi_2 = mini_df["PDI_INDEX"][id_sharpe_2] # getting weights for period

            weights_sharpe_2_performance_w.append(port_max_sharpe_weights_2)   # saving weights for periods
            port_max_ret_period_sharpe_2 = port_ret(data, next_time,port_max_sharpe_weights_2)  # calculating return for periods 
            sharpe_2_performance_w.extend(port_max_ret_period_sharpe_2) # saving return 
            pdi_weights_sharpe_2_performance_w.append(port_max_sharpe_weights_pdi_2) # calculating pdi for first period

            ################################################################## Equal Weigths Portoflio ##########################################################################

            equal_weights = PDI_DF.iloc[0]["weights"] # getting weights for period
            equal_weights_pdi = PDI_DF.iloc[0]["PDI_INDEX"] # getting weights for period
            

            weights_equal_performance.append(equal_weights)  # saving weights for periods
            port_max_ret_period_equal = port_ret(data, next_time,equal_weights) # calculating return for periods 
            equal_performance.extend(port_max_ret_period_equal) # saving return 
            pdi_weights_equal_performance.append(equal_weights_pdi) # calculating pdi for first period






        performance_frame = pd.DataFrame()
        performance_frame["Time"] = weeks_list
        performance_frame["Equal Weights"] = equal_performance
        performance_frame["Max PDI Weights"] = pdi_performance_w
        performance_frame["Max Sharpe Ratio Weights"] = sharpe_performance_w
        performance_frame["Equal Weights Cummulative"] = performance_frame["Equal Weights"].cumsum(axis=0)
        performance_frame["Max PDI Weights Cummulative"] = performance_frame["Max PDI Weights"].cumsum(axis=0) # cummulative returns max pdi
        performance_frame["Max Sharpe Ratio Weights Cummulative"] = performance_frame["Max Sharpe Ratio Weights"].cumsum(axis=0) #cummulative return sharpe ratio

        weights_frame = pd.DataFrame()
        weights_frame["Period"] = periods_weights
        weights_frame["Weights Max PDI"] = weights_pdi_performance_w
        weights_frame["Weights Max sharpe"] = weights_sharpe_performance_w
        weights_frame["Weights Equal"] = weights_equal_performance
        weights_frame["Weights Max PDI - PDI"] = pdi_weights_pdi_performance_w
        weights_frame["Weights Max sharpe - PDI"] = pdi_weights_sharpe_performance_w
        weights_frame["Weights Equal - PDI"] = pdi_weights_equal_performance
        weights_frame["Assets"] = assets







        return performance_frame, weights_frame



In [35]:
test = pd.read_csv("weeklyReturns.csv", index_col="Date")
test.index = pd.to_datetime(test.index).to_period("Q")
test[test.index == "2015Q4"]

,SPY,IVV,VTI,VOO,QQQ,VEA,IEFA,AGG,VWO,IEMG,...,SMN,EFU,RXD,EEH,SBM,SZK,SIJ,SCC,FUE,LD
Date,,,,,,,,,,,,,,,,,,,,,
2015Q4,0.040599,0.040475,0.041329,0.040749,0.038031,0.053311,0.052642,0.001783,0.073436,0.072951,...,-0.172486,-0.095621,-0.037584,0.036313,-0.092381,-0.071649,-0.093028,-0.055285,0.009485,0.000000
2015Q4,-0.000602,-0.000748,-0.002335,-0.000875,0.002840,-0.001864,-0.003058,0.004198,-0.001971,0.001168,...,-0.025035,-0.003705,0.004993,0.010782,-0.023435,-0.031950,0.008879,-0.030766,0.016107,0.000000
2015Q4,0.012846,0.013176,0.011018,0.012369,0.015010,0.009341,0.008300,-0.001000,-0.001410,-0.000233,...,0.025678,-0.016613,-0.034954,0.020000,-0.003582,-0.042435,-0.022464,-0.000222,0.011889,0.087651
2015Q4,0.035174,0.035025,0.032886,0.035844,0.060454,0.014014,0.013779,-0.002638,-0.003390,-0.003268,...,-0.073699,-0.030257,-0.079243,0.023529,0.000000,-0.015667,-0.060886,-0.067717,-0.019582,0.000000
2015Q4,0.006748,0.006996,0.007843,0.006680,0.008770,-0.004954,-0.004943,-0.005435,0.013039,0.015457,...,-0.019435,0.009881,-0.034145,-0.003831,-0.038102,0.025466,-0.024722,-0.009526,0.000000,-0.073893
2015Q4,-0.012455,-0.012619,-0.012970,-0.012910,-0.016345,-0.012054,-0.011354,-0.005330,-0.034415,-0.036900,...,0.042118,0.024717,0.039694,-0.011538,0.023543,0.019956,0.008018,0.000000,-0.005326,0.000000
2015Q4,0.004766,0.004595,0.003379,0.004885,0.004331,0.002918,0.003230,0.003695,0.004347,0.002874,...,-0.047548,-0.008543,-0.034400,-0.003243,0.000730,0.000000,-0.018219,0.007935,-0.006693,-0.038921
2015Q4,0.002827,0.003145,0.005332,0.003345,0.004576,0.000000,-0.001252,0.001473,0.002597,0.007402,...,0.002184,0.002281,0.010502,0.014964,-0.016417,-0.046957,-0.012023,-0.002147,-0.028302,0.000000
2015Q4,-0.003774,-0.003753,-0.003908,-0.004063,0.002628,-0.001322,0.000896,0.002397,-0.024461,-0.022517,...,0.011830,0.001517,-0.017934,-0.005769,0.000000,0.022810,0.026984,-0.009085,0.062413,-0.288400


In [44]:
np.random.seed(42)
random.seed(42)


dicti = {"MST": ['SMH', 'SPMD', 'SLYG', 'RPV', 'XSD', 'FIW', 'XTN', 'EZM', 'EWX', 'VIOG', 'XSW', 'KIE', 'PJP', 'PRN', 'XSMO', 'KCE'], "PDI_Boot":['IWO', 'IHI', 'VIOG', 'PSJ', 'XMHQ', 'BBP', 'ARKW', 'DWAS', 'FYC', 'FEMS', 'XTN', 'KRE', 'IYH', 'IAK', 'SIZE', 'IAT']}
result = {}
for uni in list(dicti.keys()):
    result[uni] = {}
    #Getting Data
    returns_weekly = pd.read_csv("weeklyReturns.csv", index_col="Date") # loading returns dataframe
    returns_weekly = returns_weekly[dicti[uni]]
    returns_weekly.index = pd.to_datetime(returns_weekly.index) # converting returns dataframe to datetime
    #Defining training data
    train_return = returns_weekly[returns_weekly.index.year <= 2015] # training on data from 2015

    for i in [6]:
        result[uni][i] = {}
        for ii in ["Sharpe Ratio"]:
            result[uni][ii] = {}
            print("---------------------- Calculating diversification and performnce for training period -------------------------------")
            pdi_train = calculate_pdi(num_assets = i, tickers = list(train_return.columns) , weekly_returns = train_return) # training on data from 2015 - getting max PDI portflio

            id_index = pdi_train[ii].idxmax() # getting index id for max pdi
            id_index_pdi = pdi_train["PDI_INDEX"][id_index]
            assets_port = ast.literal_eval(pdi_train.loc[id_index]["Assets"]) # max pdi portfolio
            print("---------------------- Selcted Portfolio -------------------------------")
            print(assets_port)
            print("------------------------------------------------------------------------")
            ini_porti = assets_port # starting portfolio 
            test_retuns = returns_weekly # defining test data 
            test_week = test_retuns[assets_port] #returns for selected portfolio
            print("-----------------------------------  Calculating Performance of selected portfolio -----------------------------------")
            performance, weights = pca_per_weights_rolling(return_data = test_week, portfolio = ini_porti, interval = "Q", ret_range_mean = 52, pdi_max_train=id_index_pdi) # running strategy for 2016 and forward
            result[uni][ii]["performance"] = performance
            result[uni][ii]["weights"] = weights
            print("----------------------------------- Done -----------------------------------")
            print("----------------------------------- Access dataframes - performance and weights  -----------------------------------")

7380it [00:19, 379.79it/s]
19996it [00:32, 614.46it/s]
19997it [00:31, 633.62it/s]
19998it [00:32, 610.52it/s]
19997it [00:28, 690.15it/s]
19999it [00:31, 635.76it/s]
19991it [00:31, 629.77it/s]
19996it [00:36, 555.34it/s]
19996it [00:33, 591.65it/s]
19998it [00:31, 631.03it/s]
19999it [00:31, 642.02it/s]
19997it [00:30, 660.95it/s]
19999it [00:31, 626.91it/s]
19999it [00:31, 628.71it/s]
19996it [00:30, 662.90it/s]
19998it [00:30, 650.12it/s]
19997it [00:29, 673.10it/s]
19992it [00:31, 625.37it/s]
19995it [00:30, 653.95it/s]
19997it [00:30, 645.21it/s]
19997it [00:32, 614.61it/s]
7336it [00:18, 394.19it/s]
19998it [00:31, 638.12it/s]
20000it [00:29, 685.97it/s]
19998it [00:28, 702.72it/s]
19999it [00:28, 704.93it/s]
19997it [00:31, 633.83it/s]
19996it [00:31, 626.55it/s]
19997it [00:32, 613.89it/s]
19997it [00:30, 649.35it/s]
19998it [00:32, 612.07it/s]
19993it [00:34, 575.19it/s]
19995it [00:33, 605.63it/s]
19997it [00:29, 667.36it/s]
19997it [00:30, 664.20it/s]
19999it [00:29, 677.39

---------------------- Calculating diversification and performnce for training period -------------------------------
---------------------- Selcted Portfolio -------------------------------
['SPMD', 'PJP', 'XSW', 'SLYG', 'XSD', 'KIE']
------------------------------------------------------------------------
-----------------------------------  Calculating Performance of selected portfolio -----------------------------------
2016Q1
Rolling range for calculatio: PeriodIndex(['2015Q2', '2015Q3', '2015Q4', '2016Q1'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q2
Rolling range for calculatio: PeriodIndex(['2015Q3', '2015Q4', '2016Q1', '2016Q2'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q3
Rolling range for calculatio: PeriodIndex(['2015Q4', '2016Q1', '2016Q2', '2016Q3'], dtype='period[Q-DEC]', name='Date', freq='Q-DEC') - period of return: 2016Q4
Rolling range for calculatio: PeriodIndex(['2016Q1', '2016Q2', '2016Q3', '2016Q4']

In [13]:
result.keys()

dict_keys(['MST', 'PDI_Boot'])

In [45]:
result["MST"]["Sharpe Ratio"]["weights"]

,Period,Weights Max PDI,Weights Max sharpe,Weights Equal,Weights Max PDI - PDI,Weights Max sharpe - PDI,Weights Equal - PDI,Assets
0,2016Q1,"[0.15, 0.14, 0.18, 0.1, 0.17, 0.26]","[0.01, 0.03, 0.39, 0.03, 0.3, 0.25]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.805464,1.272570,2.430644,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
1,2016Q2,"[0.18, 0.14, 0.17, 0.15, 0.15, 0.22]","[0.11, 0.01, 0.07, 0.0, 0.27, 0.54]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.637748,1.093798,2.149002,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
2,2016Q3,"[0.18, 0.16, 0.16, 0.13, 0.15, 0.22]","[0.2, 0.02, 0.08, 0.04, 0.02, 0.64]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.541486,1.007275,2.242779,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
3,2016Q4,"[0.18, 0.19, 0.14, 0.1, 0.16, 0.23]","[0.17, 0.02, 0.01, 0.06, 0.71, 0.04]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.331446,1.000462,1.908137,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
4,2017Q1,"[0.18, 0.2, 0.16, 0.16, 0.16, 0.14]","[0.03, 0.01, 0.02, 0.09, 0.45, 0.39]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.237045,1.034129,1.820023,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
5,2017Q2,"[0.05, 0.19, 0.24, 0.12, 0.15, 0.24]","[0.01, 0.02, 0.18, 0.03, 0.38, 0.39]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.372429,1.156584,1.855631,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
6,2017Q3,"[0.14, 0.21, 0.06, 0.06, 0.21, 0.33]","[0.02, 0.02, 0.13, 0.0, 0.47, 0.36]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.476200,1.075513,1.975065,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
7,2017Q4,"[0.16, 0.21, 0.12, 0.02, 0.21, 0.28]","[0.05, 0.02, 0.22, 0.01, 0.22, 0.48]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.810964,1.399306,2.325808,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
8,2018Q1,"[0.18, 0.14, 0.21, 0.18, 0.08, 0.21]","[0.03, 0.01, 0.54, 0.0, 0.13, 0.29]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.832057,1.110027,1.770556,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"
9,2018Q2,"[0.24, 0.19, 0.1, 0.15, 0.12, 0.21]","[0.02, 0.09, 0.59, 0.13, 0.0, 0.16]","[0.16666666666666666, 0.16666666666666666, 0.1...",2.685818,1.007798,1.381738,"[SPMD, PJP, XSW, SLYG, XSD, KIE]"


In [46]:

for i in ['MST', 'PDI_Boot']:
    for ii in ["Sharpe Ratio"]:
        result[i][ii]['performance'].to_csv(f"{i}_{ii}_performance.csv")
        result[i][ii]['weights'].to_csv(f"{i}_{ii}_weights.csv")



In [38]:
w_5_performance = performance.copy() 
w_5_weights= weights.copy() 


In [39]:
w_5_performance.to_csv("w_5_performance.csv")
w_5_weights.to_csv("w_5_weights.csv")


In [16]:
px.line(performance, x="Time", y = ["Equal Weights Cummulative",	"Max PDI Weights Cummulative",	"Max Sharpe Ratio Weights Cummulative"])

In [20]:
performance[["Equal Weights","Max PDI Weights"]]

,Equal Weights,Max PDI Weights
0,-0.050139,-0.050139
1,-0.083792,-0.083792
2,-0.029464,-0.029464
3,0.004288,0.004288
4,0.002231,0.002231
...,...,...
252,0.007403,0.008818
253,0.018238,0.013411
254,0.025029,0.029368
255,0.016052,0.014325


In [22]:
def meanRetAn(data):             
    Result = 1
    
    for i in range(len(data.index)):
        Result *= (1+data.iloc[i,:])
        
    Result = Result**(1/float(len(data.index)/52))-1
     
    return(Result)

In [27]:
performance

,Time,Equal Weights,Max PDI Weights,Max Sharpe Ratio Weights,Equal Weights Cummulative,Max PDI Weights Cummulative,Max Sharpe Ratio Weights Cummulative
0,2016-01-06,-0.050139,-0.050139,-0.050139,-0.050139,-0.050139,-0.050139
1,2016-01-13,-0.083792,-0.083792,-0.083792,-0.133931,-0.133931,-0.133931
2,2016-01-20,-0.029464,-0.029464,-0.029464,-0.163396,-0.163396,-0.163396
3,2016-01-27,0.004288,0.004288,0.004288,-0.159108,-0.159108,-0.159108
4,2016-02-03,0.002231,0.002231,0.002231,-0.156877,-0.156877,-0.156877
...,...,...,...,...,...,...,...
252,2020-12-02,0.007403,0.008818,0.014831,0.962349,0.926892,1.261700
253,2020-12-09,0.018238,0.013411,0.032117,0.980587,0.940303,1.293817
254,2020-12-16,0.025029,0.029368,0.038595,1.005616,0.969671,1.332412
255,2020-12-23,0.016052,0.014325,0.038085,1.021668,0.983996,1.370497


In [34]:
meanRetAn(performance[["Equal Weights",	"Max PDI Weights","Max Sharpe Ratio Weights"]])/(performance[["Equal Weights",	"Max PDI Weights","Max Sharpe Ratio Weights"]].std(axis=0)*np.sqrt(52))

Equal Weights               0.813877
Max PDI Weights             0.782518
Max Sharpe Ratio Weights    0.987039
dtype: float64

In [33]:
performance[["Equal Weights",	"Max PDI Weights","Max Sharpe Ratio Weights"]].std(axis=0)*np.sqrt(52)

Equal Weights               0.237679
Max PDI Weights             0.236328
Max Sharpe Ratio Weights    0.270476
dtype: float64